Code to train nnUnetV2

#nnU-net
##Imports and check GPU access



In [ ]:
import os
import shutil
from collections import OrderedDict
import zipfile
import json
import matplotlib.pyplot as plt
import nibabel as nib
from pathlib import Path
import numpy as np
import torch


In [ ]:
assert torch.cuda.is_available()

In [ ]:
# prompt: extract file in /content/drive/My Drive/Colab/nnUNet_raw/Dataset002_FullTraining/imagesTr

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab/nnUNet_raw/Dataset002_FullTraining/imagesTr/Totalsegmentator_dataset_v201.zip", 'r')
zip_ref.extractall("/content/drive/My Drive/Colab/nnUNet_raw/Dataset002_FullTraining/imagesTr")
zip_ref.close()


##Install nn-Unet

In [ ]:
!pip install nnunetv2


In [ ]:
import nnunetv2


In [ ]:
!nnUNet_print_available_pretrained_models

#Install Weights&Biases

In [ ]:
pip install wandb


#Mount Drive and prep folders


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab")
base_dir = os.getcwd()

In [ ]:
#create folders
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input:
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder
    """
    if os.path.exists(folder_path):

        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

#Environment Variables

In [ ]:
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw" : os.path.join(mount_dir, "nnUNet_raw"),
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    "nnUNet_results" : os.path.join(mount_dir, "nnUNet_results"),
    "RAW_DATA_PATH" : os.path.join(mount_dir, "RawData"), # This is used here only for convenience (not necessary for nnU-Net)!
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward. =)")

#Get nnUnet parameters

Preprocess dataset to get network config

In [ ]:

!nnUNetv2_plan_and_preprocess -d 2 --verify_dataset_integrity


In [ ]:
!nnUNetv2_train 1 3d_fullres 0 -tr nnUNetTrainer --c



Find best config from trained models after training

In [ ]:
!nnUNetv2_find_best_configuration 1 -c 3d_fullres


In [ ]:
!nnUNetv2_predict -i "/content/drive/My Drive/Colab/nnUNet_raw/Dataset001_TestSet/imagesTest" -o "/content/drive/My Drive/Colab/nnUNet_raw/Dataset001_TestSet/inference" -d 1 -c 3d_fullres -f 0
